# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

****
# My Anwers to the Exercises

In [12]:
# Importing Modules and reading in the data
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

In [13]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()

****
# Exercise1: 10 countries with the lowest infant mortality rates

In [14]:
countries = []      # list to hold country names
mortalities = []    # list to hold infant mortalities

# Find elements tagged with 'country', iterate over all such elements
for country in root.getiterator(tag='country'):
    
    # Add country name to the list
    countries.append(country.find('name').text)
    
    # Add infant mortality to the list
    mort = country.find('infant_mortality')
    if mort == None:           #if data don't exist, insert None
        mortality = None
    else:
        mortality = float(mort.text)     #if data exist, convert str to float
    mortalities.append(mortality)

In [27]:
# Create DataFrame with the above list, list lowest top10 countries
d = {'country': countries, 'infant_mortality': mortalities}
imort = pd.DataFrame(d)
d1 = imort.sort_values('infant_mortality', ascending=True).head(10)
d1['Rank'] = [i for i in range(1, 11)]
d1

,country,infant_mortality,Rank
38,Monaco,1.81,1
98,Japan,2.13,2
117,Bermuda,2.48,3
36,Norway,2.48,4
106,Singapore,2.53,5
37,Sweden,2.60,6
10,Czech Republic,2.63,7
78,Hong Kong,2.73,8
79,Macao,3.13,9
44,Iceland,3.15,10


****
# Exercise2: 10 cities with the largest population

In [16]:
cities = []         # list to hold city names
populations = []    # list to hold city populations

# Find elements tagged with 'city', iterate over all such elements
for city in root.getiterator(tag='city'):
    
    # Add country name to the list    
    cities.append(city.find('name').text)
    
    # Obtain the latest population information
    years = []    # list to hold years that population data are available
    pops = {}     # dict to hold populations for those years
    for pop in city.findall('population'):
        year = pop.get('year')
        years.append(int(year))
        pops[year] = int(pop.text)
    
    # If data available, get the latest, if not, insert 0
    if years != []:
        populations.append(pops[str(max(years))])
    else:
        populations.append(None)

In [17]:
# Create DataFrame with the above list, list largest top10 cities
d2 = {'City': cities, 'Population': populations}
city_populations = pd.DataFrame(d2).dropna()     # Drop data points with None (NaN)
city_populations = city_populations.sort_values('Population', ascending=False).head(10)
city_populations['Rank'] = [i for i in range(1, 11)]
city_populations

,City,Population,Rank
1341,Shanghai,22315474.0,1
771,Istanbul,13710512.0,2
1527,Mumbai,12442373.0,3
479,Moskva,11979529.0,4
1340,Beijing,11716620.0,5
2810,São Paulo,11152344.0,6
1342,Tianjin,11090314.0,7
1064,Guangzhou,11071424.0,8
1582,Delhi,11034555.0,9
1067,Shenzhen,10358381.0,10


****
# Exercise3: 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [18]:
data = {}

for country in root.getiterator(tag='country'):
    country_name = country.find('name').text
    years = []
    pops = {}
    for pop in country.findall('population'):
        year = pop.get('year')
        years.append(int(year))
        pops[year] = int(pop.text)
    if years != []:
        country_population = pops[str(max(years))]
    else:
        country_population = 0
    
    egroup = country.findall('ethnicgroup')
    for e in egroup:
        if e.text not in data.keys():
            data[e.text] = 0
        data[e.text] += country_population * float(e.get('percentage')) / 100

In [19]:
ethnic_data = pd.Series(data)
d2 = (ethnic_data/100000).round(1).sort_values(ascending=False).head(10)
d2= d2.to_frame('Population (millions)')
d2['Rank'] = [i for i in range(1, 11)]
d2

,Population (millions),Rank
Han Chinese,12450.6,1
Indo-Aryan,8718.2,2
European,4948.7,3
African,3183.3,4
Dravidian,3027.1,5
Mestizo,1577.3,6
Bengali,1467.8,7
Russian,1318.6,8
Japanese,1265.3,9
Malay,1219.9,10


****
# Exercise4: name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [20]:
# This time, I dealt with try/except to deal with data that are missing/not available
# Besides that, the approach here is the same as above

In [21]:
# a) Longest Rivers
rivers = []
for river in root.getiterator(tag='river'):
    try:
        river_name = river.find('name').text
    except:
        river_name = None
        
    try:
        length = int(river.find('length').text)
    except:
        length = 0
    try:
        country = river.get('country')
    except:
        country = None
    rivers.append({'River name': river_name, 'country': country, 'Length (km)': length})

In [22]:
river_data = pd.DataFrame(rivers)
d3a = river_data.sort_values('Length (km)', ascending=False).head(10)
d3a['Rank'] = [i for i in range(1, 11)]
d3a

,Length (km),River name,country,Rank
174,6448,Amazonas,CO BR PE,1
137,6380,Jangtse,CN,2
136,4845,Hwangho,CN,3
123,4400,Lena,R,4
205,4374,Zaire,RCB ZRE,5
138,4350,Mekong,CN LAO THA K VN,6
115,4248,Irtysch,R KAZ CN,7
186,4184,Niger,RMM RN WAN RG,8
160,4130,Missouri,USA,9
119,4092,Jenissej,R,10


In [23]:
# b) Largest Lakes
lakes = []
for lake in root.getiterator(tag='lake'):
    try:
        lake_name = lake.find('name').text
    except:
        lake_name = None
        
    try:
        area = int(lake.find('area').text)
    except:
        area = 0
    try:
        country = lake.get('country')
    except:
        country = None
    lakes.append({'Lake name': lake_name, 'country': country, 'Area (km^2)': area})

In [24]:
lake_data = pd.DataFrame(lakes)
d3b = lake_data.sort_values('Area (km^2)', ascending=False).head(10)
d3b['Rank'] = [i for i in range(1, 11)]
d3b

,Area (km^2),Lake name,country,Rank
54,386400,Caspian Sea,R AZ KAZ IR TM,1
109,82103,Lake Superior,CDN USA,2
81,68870,Lake Victoria,EAT EAK EAU,3
106,59600,Lake Huron,CDN USA,4
108,57800,Lake Michigan,USA,5
47,41650,Dead Sea,IL JOR WEST,6
83,32893,Lake Tanganjika,ZRE Z BI EAT,7
98,31792,Great Bear Lake,CDN,8
43,31492,Ozero Baikal,R,9
89,29600,Lake Malawi,MW MOC EAT,10


In [25]:
# c) Highest airports
airports = []
for ap in root.getiterator(tag='lake'):
    try:
        ap_name = ap.find('name').text
    except:
        ap_name = None
        
    try:
        elevation = int(ap.find('elevation').text)
    except:
        area = 0
    try:
        country = ap.get('country')
    except:
        country = None
    airports.append({'Airport name': ap_name, 'Country': country, 'Elevation (m)': elevation})

In [26]:
ap_data = pd.DataFrame(airports)
d4b = ap_data.sort_values('Elevation (m)', ascending=False).head(10)
d4b['Rank'] = [i for i in range(1, 11)]
d4b

,Airport name,Country,Elevation (m),Rank
134,Licancabur Crater Lake,RCH BOL,5600,1
64,Nam Co,CN,4718,2
131,Lake Titicaca,BOL PE,3812,3
132,Poopo,BOL,3686,4
133,Salar de Uyuni,BOL,3650,5
58,Koli Sarez,TAD,3250,6
128,Lake Irazu,CR,3200,7
63,Qinghai Lake,CN,3195,8
62,Segara Anak,RI,2008,9
118,Lake Tahoe,USA,1900,10
